In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [6]:
response = llm.invoke("Hello! how are you?")

print(response.content)

I am doing well, thank you for asking!  How are you today?


In [7]:
PDF_TO_TEXT_PROMPT = """
You are a highly accurate document parser. Your task is to extract *everything* from a PDF document.

The input is raw PDF content (converted to text and image snippets). Extract and preserve the following:

1. **All textual content**, including:
   - Page numbers
   - Headers and footers
   - Paragraphs, tables, lists, and titles
   - Questions and answers (if present)
2. **Image content** should be included as summaries or captions like:
   - [Image]: A diagram showing a neural network architecture.
3. Preserve the **page structure** with clear delimiters like:
   - === Page 1 ===
   - === Page 2 ===

Make sure your output is:
- In Markdown
- Preserves the order and layout as much as possible
- Suitable for semantic chunking and retrieval-based systems

Do not skip any content. Your goal is to represent the document *completely and accurately* for storage in a vector database.

Now extract the content.
"""

In [8]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("resources/HSC26-Bangla1st-Paper-4.pdf")
# documents = loader.load()

In [9]:
import base64

with open("resources/HSC26-Bangla1st-Paper-4.pdf", "rb") as f:
    pdf_bytes = f.read()
    pdf_base64 = base64.b64encode(pdf_bytes).decode()

In [41]:
messages = [
    SystemMessage(content=PDF_TO_TEXT_PROMPT),
    HumanMessage(content=[
        {
            "type": "text",
            "text": "Please extract and process all text from this PDF document, paying special attention to Bangla/Bengali content."
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:application/pdf;base64,{pdf_base64}"
            }
        }
    ])
]

In [45]:
# import time

# start_time = time.time()
# response = llm.invoke(messages)
# processing_time = time.time() - start_time


# print(processing_time)

In [46]:
# from langchain_core.messages import SystemMessage, HumanMessage

# extracted_texts = []
# for doc in documents:
#     prompt = [
#         SystemMessage(content=PDF_TO_TEXT_PROMPT),
#         HumanMessage(doc.page_content)
#     ]
#     response = llm.invoke(prompt)
#     extracted_texts.append(response.content)

# # Combine extracted texts
# bangla_text = "\n".join(extracted_texts)

In [24]:
from IPython.display import display
display(bangla_text[:500])

'শব্দার্ব ও টীকা\nেূ  শব্দ শলব্দি অর্ব ও িযাখ্যা\nমনু-সংহীতা র্ব্ধানকতিা ব্া িাস্ত্রপ্রকণতা মুর্নর্ব্কিষ।\nমনু-সংহীতা মনু-প্রণীত মানুকষি আচিণর্ব্র্ধ সংক্রান্ত গ্রন্থ ।\nপ্রিাপর্ত িীকব্ি রষ্ট্া। ব্রহ্মা। ইর্ন র্ব্ক ি যদব্তা।\nপঞ্চিি মদনকদকব্ি ব্যব্হােি পঁাচ ধিকনি ব্াণ।\nকন্সটি নানা িকম ব্াদযেকন্ত্রি ঐকতান।\nযসকিা স্বণিকাি, যসানাি অলংকাি প্রস্তুতকািক\nব্ব্িি যকালাহকলি মি হস্তী দ্বািা \nসংগীতসিস্বতীি পদ্মব্ন\nদজলত র্ব্দজলত কর্ি া আর্ম যতা \nর্ব্ব্াহ-ব্ার়্েকত র্গ া\nউঠিলাম\nঅনুপম র্নকিি র্ব্ব্াহোত্রাি পর্িজির্ত'

## Google SDK

In [21]:
import google.generativeai as genai
from langsmith import traceable
import time

gemini_model = genai.GenerativeModel('gemini-1.5-flash')
uploaded_file = genai.upload_file("resources/HSC26-Bangla1st-Paper-15-20.pdf", mime_type="application/pdf")

In [22]:
PDF_EXTRACTION_PROMPT = """
You are a highly accurate document parsing AI. Your task is to extract text — **especially all Bangla (Bengali)** — from a PDF document exactly as it appears, without missing any content, while preserving the full layout and formatting.

Follow these strict instructions carefully:

1. 🔹 **DO NOT skip any words**, especially those near images or tables. Extract all content **faithfully** without summarizing or interpreting.
2. 🔹 **Preserve full structure**, including:
   - Headings
   - Paragraphs
   - Bullet points
   - Numbered lists
   - Tables (use Markdown table format)
   - Questions and answers
   - Page number (from the actual document body, not watermark)
3. 🔹 **If there is an image**, include a line like:
   `[Image]: <brief description>` in its position.
   Do **not skip** any text that appears beside or around the image.
4. 🔹 For **tables**, maintain the full structure using proper Markdown tables. If the table includes images, use placeholders like `[Image in Cell]: <description>`.
5. 🔹 Begin every page with:
   `=== Page {start_page_number + i} ===`
   where `{start_page_number + i}` is the actual visible page number shown in the document.
6. 🔹 If you detect repeated headers or footers (such as watermarks, website names, or page footnotes repeated on every page), **do not include them** in the output. Only extract meaningful document body content.
7. 🔹 Keep **Bangla text as-is**, and leave any English words unchanged.

📌 Output the extracted content in clean **Markdown** format — do not include extra explanations, metadata, or summaries. Your job is only to extract, not interpret.

Now begin extracting the content accurately.
"""


In [23]:
@traceable()
def extract_text():
    start_time = time.time()
    response = gemini_model.generate_content([PDF_EXTRACTION_PROMPT, uploaded_file])
    processing_time = time.time() - start_time
        
    genai.delete_file(uploaded_file.name)
    
    print(processing_time)
    return response

result = extract_text()

86.26944756507874


In [24]:
extracted_text = result.text

print(f"Total character = {len(extracted_text)}")

Total character = 11418


In [25]:
from IPython.display import display, Markdown

display(Markdown(extracted_text))

=== Page 15 ===
HSC 26
অনলাইন ব্যাচ
বাংলা ইংরেজি আইসিটি
MINUTE
SCHOOL
না যেটা তাহাকে ছাড়াইয়া বিশেষ করিয়া চোখে পড়িতে পারে। সে নিজের চারি দিকের সকলের চেয়ে অধিক -
রজনীগন্ধার শুভ্র মঞ্জরীর মতো সরল বৃন্তটির উপরে দাঁড়াইয়া, যে গাছে ফুটিয়াছে সে গাছকে সে একেবারে
অতিক্রম করিয়া উঠিয়াছে। সঙ্গে দুটি-তিনটি ছোটো ছোটো মেয়ে ছিল, তাহাদিগকে লইয়া তাহার হাসি এবং
কথার আর অন্ত ছিল না। আমি হাতে একখানা বই লইয়া সে দিকে কান পাতিয়া রাখিয়াছিলাম। যেটুকু কানে
আসিতেছিল সে তো সমস্তই ছেলেমানুষদের সঙ্গে ছেলেমানুষি কথা। তাহার বিশেষত্ব এই যে, তাহার মধ্যে
বয়সের তফাত কিছুমাত্র ছিল না- ছোটোদের সঙ্গে সে অনায়াসে এবং আনন্দে ছোটো হইয়া গিয়াছিল। সঙ্গে
কতকগলি ছবিওয়ালা ছেলেদের গল্পের বই-তাহারই কোন একটা বিশেষ গল্প শোনাইবার জন্য মেয়েরা তাহাকে
ধরিয়া পড়িল। এ গল্প নিশ্চয় তারা বিশ-পঁচিশ বার শনিয়াছে। মেয়েদের কেন যে এত আগ্রহ তাহা বুঝিলাম।
সেই সুধাকণ্ঠের সোনার কাঠিতে সকল কথা যে সোনা হইয়া ওঠে। মেয়েটির সমস্ত শরীর মন যে একেবারে
প্রাণে ভরা, তার সমস্ত চলায় বলায় স্পর্শে প্রাণ ঠিকরিয়া ওঠে।
তাই মেয়েরা যখন তার মুখে গল্প শোনে তখন, গল্প নয়, তাহাকেই শোনে; তাহাদের হৃদয়ের উপর প্রাণের ঝর্না
ঝরিয়া পড়ে।
তার সেই উদ্ভাসিত প্রাণ আমার
সেদিনকার সমস্ত সূর্যকিরণকে সজীব করিয়া
তুলিল; আমার মনে হইল, আমাকে যে প্রকৃতি
তাহার আকাশ দিয়া বেষ্টন করিয়াছে সে ঐ
তরুণীরই অক্লান্ত অম্লান প্রাণের বিশ্বব্যাপী
বিস্তার।-পরের স্টেশনে পৌঁছিতেই খাবারওয়ালাকে ডাকিয়া সে খুব খানিকটা চানা-মুঠ কিনিয়া লইল এবং
মেয়েদের সঙ্গে মিলিয়া নিতান্ত ছেলেমানুষের মতো করিয়া কলহাস্য করিতে করিতে অসংকোচে খাইতে
লাগিল। আমার প্রকৃতি যে জাল দিয়া বেড়া—আমি কেন বেশ সহজে হাসিমুখে মেয়েটির কাছে এই চানা
একমুঠো চাহিয়া লইতে পারিলাম না। হাত বাড়াইয়া দিয়া কেন আমার লোভ স্বীকার করিলাম না।
মা ভালো-লাগা এবং মন্দ-লাগার মধ্যে দোমনা হইয়া ছিলেন। গাড়িতে আমি পুরুষমানুষ, তবু ইহার কিছুমাত্র
সংকোচ নাই, বিশেষত এমন লোভীর মতো খাইতেছে, সেটা ঠিক তাঁর পছন্দ হইতেছিল না; অথচ ইহাকে
বেহায়া বলিয়াও তাঁর ভ্রম হয় নাই। তাঁর মনে হইল, এ মেয়ের বয়স হইয়াছে কিন্তু শিক্ষা হয় নাই। মা হঠাৎ
কারও সঙ্গে আলাপ করিতে পারেন না। মানুষের সঙ্গে দূরে দূরে থাকাই তাঁর অভ্যাস। এই মেয়েটির পরিচয়
লইতে তাঁর খুব ইচ্ছা, কিন্তু স্বাভাবিক বাধা কাটাইয়া উঠিতে পারিতেছিলেন না।
এমন সময়ে গাড়ি একটা বড়ো স্টেশনে আসিয়া থামিল। সেই জেনারেল-সাহেবের একদল অনুসঙ্গী এই স্টেশন
হইতে উঠিবার উদ্‌্যোগ করিতেছে। গাড়িতে কোথাও জায়গা নাই।
বারবার আমাদের গাড়ির সামনে দিয়া তারা ঘুরিয়া গেল। মা তো ভয়ে আড়ষ্ট, আমিও মনের মধ্যে শান্তি
পাইতেছিলাম না।
গাড়ি ছাড়িবার অল্পকাল-পূর্বে একজন দেশী রেলওয়ে কর্মচারী নাম-লেখা দুইখানা টিকিট গাড়ির দুই বেঞ্চের
শিয়রের কাছে লঙ্কাইয়া দিয়া আমাকে বলিল, “এ গাড়ির এই দুই বেঞ্চ আগে হইতেই দুই সাহেব রিজার্ভ
করিয়াছেন, আপনাদিগকে অন্য গাড়িতে যাইতে হইবে।”
[Image]: A picture depicting a scene from the story.
15

=== Page 16 ===
HSC 26
অনলাইন ব্যাচ
বাংলা ইংরেজি আইসিটি
MINUTE
SCHOOL
আমি তো তাড়াতাড়ি ব্যস্ত হইয়া দাঁড়াইয়া উঠিলাম। মেয়েটি হিন্দিতে বলিল, “না, আমরা গাড়ি ছাড়িব না।”
সে লোকটি রোখ করিয়া বলিল, “না ছাড়িয়া উপায় নাই।”
কিন্তু, মেয়েটির চলিষ্ণুতার কোনো লক্ষণ না দেখিয়া সে নামিয়া গিয়া ইংরেজ স্টেশন-মাস্টারকে ডাকিয়া
আনিল। সে আসিয়া আমাকে বলিল, “আমি দুঃখিত, কিন্তু-”
শুনিয়া আমি 'কুলি কুলি' করিয়া ডাক ছাড়িতে লাগিলাম। মেয়েটি উঠিয়া দুই চক্ষে অগ্নিবর্ষণ করিয়া বলিল,
“না, আপনি যাইতে পারিবেন না, যেমন আছেন বসিয়া থাকুন।”
বলিয়া সে দ্বারের কাছে দাঁড়াইয়া স্টেশনমাস্টারকে ইংরেজি ভাষায় বলিল, “এ গাড়ি আগে হইতে রিজার্ভ
করা, এ কথা মিথ্যা কথা।” বলিয়া নাম লেখা টিকিটটি খুলিয়া প্ল্যাটফর্মে ছুঁড়িয়া ফেলিয়া দিল।
বলিয়া সে দ্বারের কাছে দাঁড়াইয়া স্টেশনমাস্টারকে ইংরেজি ভাষায় বলিল, “এ গাড়ি আগে হইতে রিজার্ভ
করা, এ কথা মিথ্যা কথা।” বলিয়া নাম লেখা টিকিটটি খুলিয়া প্ল্যাটফর্মে ছুঁড়িয়া ফেলিয়া দিল।
বলিয়া সে দ্বারের কাছে দাঁড়াইয়া
স্টেশনমাস্টারকে ইংরেজি ভাষায় বলিল, “এ গাড়ি
আগে হইতে রিজার্ভ করা, এ কথা মিথ্যা কথা।”
বলিয়া নাম লেখা টিকিটটি খুলিয়া প্ল্যাটফর্মে
ছুঁড়িয়া ফেলিয়া দিল।
[Image]: A picture of a reserved railway ticket.
ইতিমধ্যে আর্দালি-সমেত ইউনিফর্ম-পরা সাহেব
RESERVEDERM BENGAL RAILWA
দ্বারের কাছে আসিয়া দাঁড়াইয়াছে। গাড়িতে সে তার আসবাব উঠাইবার জন্য আর্দালিকে প্রথমে ইশারা
করিয়াছিল। তাহার পরে মেয়েটির মুখে তাকাইয়া, তার কথা শুনিয়া, ভাব দেখিয়া, স্টেশন-মাস্টারকে একটু
স্পর্শ করিল এবং তাহাকে আড়ালে লইয়া গিয়া কী কথা হইল জানি না। দেখা গেল,
গাড়ি ছাড়িবার সময় অতীত হইলেও আর-একটা
গাড়ি জুড়িয়া তবে ট্রেন ছাড়িল।
মেয়েটি তার দলবল লইয়া আবার একপত্তন চানা-
মুঠ খাইতে শুরু করিল, আর আমি লজ্জায়
জানলার বাহিরে মুখে বাড়াইয়া প্রকৃতির শোভা দেখিতে লাগিলাম।
কানপুরে গাড়ি আসিয়া থামিল। মেয়েটি জিনিসপত্র বাঁধিয়া প্রস্তুত-স্টেশনে একটি হিন্দুস্থানি চাকর ছুটিয়া
আসিয়া ইহাদিগকে নামাইবার উদ্যোগ করিতে লাগিল।
মা তখন আর থাকিতে পারিলেন না। জিজ্ঞাসা করিলেন, “তোমার নাম কী মা।”
মেয়েটি বলিল, “আমার নাম কল্যাণী।”
শুনিয়া মা এবং আমি দুজনেই চমকিয়া উঠিলাম।
“তোমার বাবা-"
“তিনি এখানকার ডাক্তার, তাঁহার নাম শম্ভুনাথ সেন।”
তার পরেই সবাই নামিয়া গেল।
16

=== Page 17 ===
HSC 26
অনলাইন ব্যাচ
বাংলা ইংরেজি আইসিটি
10
MINUTE
SCHOOL
মামার নিষেধ অমান্য করিয়া, মাতৃ-আজ্ঞা ঠেলিয়া, তার পরে আমি কানপুরে আসিয়াছি। কল্যাণীর বাপ এবং
কল্যাণীর সঙ্গে দেখা হইয়াছে। হাত জোড় করিয়াছি, মাথা হেঁট করিয়াছি; শম্ভুনাথবাবুর হৃদয় গলিয়াছে।
কল্যাণী বলে, “আমি বিবাহ করিব না।”
আমি জিজ্ঞাসা করিলাম, “কেন।”
সে বলিল, “মাতৃ-আজ্ঞা।”
কী সর্বনাশ। এ পক্ষেও মাতুল আছে নাকি।
তার পরে বুঝিলাম, মাতৃভূমি আছে। সেই বিবাহ-ভাঙার পর হইতে কল্যাণী মেয়েদের শিক্ষার ব্রত গ্রহণ
করিয়াছে।
কিন্তু, আমি আশা ছাড়িতে পারিলাম না। সেই সুরটি যে আমার হৃদয়ের মধ্যে আজও বাজিতেছে—সে যেন
কোন ওপারের বাঁশি- আমার সংসারের বাহির হইতে আসিল - সমস্ত সংসারের বাহিরে ডাক দিল। আর, সেই-
যে রাত্রির অন্ধকারের মধ্যে আমার কানে আসিয়াছিল “জায়গা আছে”, সে যে আমার চিরজীবনের গানের
ধুয়া হইয়া রহিল। তখন আমার বয়স ছিল তেইশ, এখন হইয়াছে সাতাশ। এখনো আশা ছাড়ি নাই, কিন্তু
মাতুলকে ছাড়িয়াছি। নিতান্ত এক ছেলে বলিয়া মা আমাকে ছাড়িতে পারেন নাই।
তোমরা মনে করিতেছ, আমি বিবাহের আশা করি? না, কোনো কালেই না। আমার মনে আছে, কেবল সেই এক
রাত্রির অজানা কণ্ঠের মধুর সুরের আশা-জায়গা আছে। নিশ্চয়ই আছে। নইলে দাঁড়াব কোথায়। তাই বৎসরের
পর বৎসর যায় আমি এইখানেই আছি। দেখা হয়, সেই কণ্ঠ শুনি, যখন সুবিধা পাই কিছু তার কাজ করিয়া দিই
আর মন বলে, এই তো জায়গা পাইয়াছি। ওগো অপরিচিতা, তোমার পরিচয়ের শেষ হইল না, শেষ হইবে না;
কিন্তু ভাগ্য আমার ভালো, এই তো আমি জায়গা পাইয়াছি।
17

=== Page 18 ===
HSC 26
অনলাইন ব্যাচ
বাংলা ইংরেজি আইসিটি
লেখক পরিচিতি
নাম
প্রকৃত নাম: রবীন্দ্রনাথ ঠাকুর।
ছদ্মনাম: ভানুসিংহ ঠাকুর।
জন্ম পরিচয়
জন্ম তারিখ: ৭ মে, ১৮৬১ খ্রিষ্টাব্দ (২৫ বৈশাখ,
১২৬৮ বঙ্গাব্দ)
জন্মস্থান: জোড়াসাঁকো, কলকাতা, ভারত।
10
MINUTE
SCHOOL
বংশ পরিচয়
পিতার নাম: মহর্ষি দেবেন্দ্রনাথ ঠাকুর।
মাতার নাম: সারদা দেবী।
পিতামহের নাম: প্রিন্স দ্বারকানাথ ঠাকুর।
শিক্ষাজীবন
রবীন্দ্রনাথ বাল্যকালে ওরিয়েন্টাল সেমিনারি, নর্মাল স্কুল, বেঙ্গল একাডেমি, সেন্ট
জেভিয়ার্স স্কুল প্রভৃতি শিক্ষা প্রতিষ্ঠানে অধ্যয়ন করলেও স্কুলের পাঠ শেষ করতে
পারেননি। ১৭ বছর বয়সে ব্যারিস্টারি পড়তে ইংল্যান্ড গেলেও কোর্স সম্পন্ন করা
সম্ভব হয়নি। তবে গৃহশিক্ষকের কাছ থেকে জ্ঞানার্জনে তাঁর কোনো ত্রুটি হয়নি।
১৮৮৪ খ্রিস্টাব্দ থেকে রবীন্দ্রনাথ তাঁর পিতার আদেশে বিষয়কর্ম পরিদর্শনে নিযুক্ত হন
এবং ১৮৯০ খ্রিস্টাব্দ থেকে দেশের বিভিন্ন অঞ্চলে জমিদারি দেখাশুনা করেন। এ
পেশা/ কর্মজীবন সূত্রে তিনি কুষ্টিয়ার শিলাইদহ ও সিরাজগঞ্জের শাহজাদপুরে দীর্ঘ সময় অবস্থান
করেন। তিনি ছিলেন অনন্য চিত্রশিল্পী, অনুসন্ধিৎসু বিশ্বপরিব্রাজক, দক্ষ সম্পাদক
এবং অসামান্য শিক্ষা-সংগঠক ও চিন্তক। নিজে প্রাতিষ্ঠানিক শিক্ষাগ্রহণে নিরুৎসাহী
হলেও 'বিশ্বভারতী' নামের বিশ্ববিদ্যালয়ের তিনি স্বাপ্নিক ও প্রতিষ্ঠাতা।
[Image]: A black and white photograph of Rabindranath Tagore.
কাব্যগ্রন্থ: মানসী, সোনার তরী, চিত্রা, চৈতালি, ক্ষণিকা, নৈবেদ্য, গীতাঞ্জলি, বলাকা,
পূরবী, পুনশ্চ, বিচিত্রা, সেঁজুতি, জন্মদিনে, শেষ লেখা প্রভৃতি বিশেষভাবে
উল্লেখযোগ্য।
সাহিত্যকর্ম
উপন্যাস: চোখের বালি, গোরা, ঘরে-বাইরে, চতুরঙ্গ, নৌকাডুবি, যোগাযোগ, রাজর্ষি,
শেষের কবিতা প্রভৃতি।
পুরষ্কার ও
সম্মাননা
নাটক: অচলায়তন, চিরকুমার সভা, ডাকঘর, মুকুট, মুক্তির উপায়, মুক্তধারা,
রক্তকরবী, রাজা প্রভৃতি।
প্রবন্ধগ্রন্থ: বিচিত্র প্রবন্ধ, শিক্ষা, কালান্তর, সভ্যতার সংকট ইত্যাদি।
ভ্রমণকাহিনী: জাপানযাত্রী, পথের সঞ্চয়, পারস্য, রাশিয়ার চিঠি, য়ুরোপ যাত্রীর
ডায়েরী, য়ুরোপ প্রবাসীর পত্র প্রভৃতি।
'গীতাঞ্জলি' এবং অন্যান্য কাব্যের কবিতার সমন্বয়ে স্বঅনূদিত Song Offerings
গ্রন্থের জন্য প্রথম এশীয় হিসেবে, নোবেল পুরস্কার (১৯১৩), কলকাতা বিশ্ববিদ্যালয়
কর্তৃক ডি-লিট (১৯১৩), ঢাকা বিশ্ববিদ্যালয় কর্তৃক ডি-লিট (১৯৩৬), অক্সফোর্ড
বিশ্ববিদ্যালয় কর্তৃক ডি-লিট (১৯৪০)।
জীবনাবসান
৭ আগস্ট, ১৯৪১ খ্রিস্টাব্দ (২২ শ্রাবণ, ১৩৪৮ বঙ্গাব্দ), জোড়াসাঁকোর ঠাকুরবাড়িতে।
18

=== Page 19 ===
HSC 26
অনলাইন ব্যাচ
বাংলা ইংরেজি আইসিটি
পাঠ পরিচিতি
“অপরিচিতা” প্রথম প্রকাশিত হয় প্রমথ চৌধুরী সম্পাদিত মাসিক 'সবুজপত্র' পত্রিকার ১৩২১ বঙ্গাব্দের
(১৯১৪) কার্তিক সংখ্যায়। এটি প্রথম গ্রন্থভুক্ত হয় রবীন্দ্রগল্পের সংকলন ‘গল্পসপ্তক'-এ এবং পরে, 'গল্পগুচ্ছ'
তৃতীয় খণ্ডে (১৯২৭)। “অপরিচিতা” গল্পে অপরিচিতা বিশেষণের আড়ালে যে বলিষ্ঠ ব্যক্তিত্বের অধিকারী
নারীর কাহিনি বর্ণিত হয়েছে, তার নাম কল্যাণী। অমানবিক যৌতুক প্রথার নির্মম বলি হয়েছে এমন নারীদের
গল্প ইতঃপূর্বে রচনা করেছিলেন রবীন্দ্রনাথ। কিন্তু এই গল্পেই প্রথম যৌতুক প্রথার বিরুদ্ধে নারী-পুরুষের
সম্মিলিত প্রতিরোধের কথকতা শোনালেন তিনি। এ গল্পে পিতা শম্ভুনাথ সেন এবং কন্যা কল্যাণীর স্বতন্ত্ৰবীক্ষা ও
আচরণে সমাজে গেড়ে-বসা ঘৃণ্য যৌতুক প্রথা প্রতিরোধের সম্মুখীন হয়েছে। পিতার বলিষ্ঠ প্রতিরোধ এবং
কল্যাণীর দেশচেতনায় ঋদ্ধ ব্যক্তিত্বের জাগরণ ও তার অভিব্যক্তিতে গল্পটি স্বার্থক। “অপরিচিতা” উত্তম
পুরুষের জবানিতে লেখা গল্প। গল্পের কথক অনুপম বিশ শতকের দ্বিতীয় দশকের যুদ্ধসংলগ্ন সময়ের সেই
বাঙালি যুবক, যে বিশ্ববিদ্যালয়ের উচ্চতর উপাধি অর্জন করেও ব্যক্তিত্বরহিত, পরিবারতন্ত্রের কাছে অসহায়
পুতুলমাত্র। তাকে দেখলে আজো মনে হয়, সে যেন মায়ের কোলসংলগ্ন শিশুমাত্র। তারই বিয়ে উপলক্ষ্যে যৌতুক
নিয়ে নারীর চরম অবমাননাকালে শম্ভুনাথ সেনের কন্যা-সম্প্রদানে অসম্মতি গল্পটির শীর্ষ মুহূর্ত। অনুপম নিজের
গল্প বলতে গিয়ে ব্যাঙ্গার্থে জানিয়ে দিয়েছে সেই অঘটন সংঘটনের কথাটি। বিয়ের লগ্ন যখন প্রস্তুত তখন কন্যার
লগ্নভ্রষ্ট হওয়ার লৌকিকতাকে অগ্রাহ্য করে শম্ভুনাথ সেনের নির্বিকার অথচ বলিষ্ঠ প্রত্যাখ্যান নতুন এক সময়ের
আশু আবির্ভাবকেই সংকেতবহ করে তুলেছে। কর্মীর ভূমিকায় বলিষ্ঠ ব্যক্তিত্বের জাগরণের মধ্য দিয়ে গল্পের
শেষাংশে কল্যাণীর শুচিশুভ্র আত্মপ্রকাশও ভবিষ্যতের নতুন নারীর আগমনীর ইঙ্গিতে পরিসমাপ্ত। 'অপরিচিতা'
মনস্তাপে ভেঙেপড়া এক ব্যক্তিত্বহীন যুবকের স্বীকারোক্তির গল্প, তার পাপস্খালনের অকপট কথামালা।
অনুপমের আত্মবিবৃতির সূত্র ধরেই গল্পের নারী কল্যাণী অসামান্য হয়ে উঠেছে। গল্পটিতে পুরুষতন্ত্রের
অমানবিকতার স্ফুরণ যেমনঘটেছে, তেমনি একই সঙ্গে পুরুষের ভাষ্যে নারীর প্রশস্তিও কীর্তিত হয়েছে।
10
MINUTE
SCHOOL
19

=== Page 20 ===
HSC 26
অনলাইন ব্যাচ
বাংলা ইংরেজি আইসিটি
পাঠ্যপুস্তকের প্রশ্ন
বহুনির্বাচনী
১। অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?
(ক) ডাক্তারি
(খ) ওকালতি
(গ) মাস্টারি
(ঘ) ব্যবসা
উত্তর: খ
২। মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার-
(ক) প্রতিপত্তি
(খ) প্রভাব
(গ) বিচক্ষণতা
(ঘ) কূট বুদ্ধি
উত্তর: খ
৩। দীপুর চাচার সঙ্গে 'অপরিচিতা' গল্পের কোন চরিত্রের মিল আছে?
(ক) হরিশের
(খ) মামার
(গ) শিক্ষকের
(ঘ) বিনুর
উত্তর: খ
ব্যাখ্যা: অনুপমের পিতার মৃত্যুর পর তার মামাই তাদের পরিবারের দায়িত্ব নেন। পরিবারে তার প্রভাবের
কথা বোঝাতেই অনুপম মামাকে 'ভাগ্যদেবতার প্রধান এজেন্ট' বলে।
নিচের উদ্দীপকটি পড়ে ৩ ও ৪ সংখ্যক প্রশ্নের উত্তর দাও।
পিতৃহীন দীপুর চাচাই ছিলেন পরিবারের কর্তা। দীপু শিক্ষিত হলেও তার সিদ্ধান্ত নেওয়ার ক্ষমতা ছিল না।
চাচা তার বিয়ের উদ্যোগ নিলেও যৌতুক নিয়ে বাড়াবাড়ি করার কারণে কন্যার পিতা অপমানিত বোধ করে
বিয়ের আলোচনা ভেঙে দেন। দীপু মেয়েটির ছবি দেখে মুগ্ধ হলেও তার চাচাকে কিছুই বলতে পারেননি।
৩। দীপুর চাচার সঙ্গে 'অপরিচিতা' গল্পের কোন চরিত্রের মিল আছে?
(ক) হরিশের
(খ) মামার
(গ) শিক্ষকের
(ঘ) বিনুর
উত্তর: খ
ব্যাখ্যা: দীপুর চাচা ও 'অপরিচিতা' গল্পের অনুপমের মামার লোভ সীমাহীন। তারা উভয়েই যৌতুকলোভী।
এই লোভী মানসিকতার দিক দিয়ে তাদের মধ্যে মিল রয়েছে।
৪। উক্ত চরিত্রে প্রাধান্য পেয়েছে-
i. দৌরাত্ম্য
ii. হীনম্মন্যতা
iii. লোভ
নিচের কোনটি ঠিক?
(ক) i, ii
(খ) ii, iii
(গ) i, iii
(ঘ) i, ii, iii
উত্তর: খ
20
